In [ ]:
from datetime import date
from pathlib import Path
import pandas as pd
import typing
import math
import os

from MktData.equity_prices import EquityPrices
from MktData.equity_vol import EquityVol
from MktData.mkt_data import MktData

from crossasset.crossasset_base import CrossAssetContext
from nxpy import pro

pd.options.display.float_format = "{:,.2f}".format

%load_ext autoreload
%autoreload 2

In [18]:
tickers = ['MXEA Index', 'RTY Index', 'SPX Index']
mktdata=MktData(tickers)

In [4]:
inforce_file = r'C:\Users\S0053071\Repos\Winterfell\2023_BudgetRecon\IUL_and_VUL_AV_and_Caps_Monthly_2023.xlsx'

In [23]:
inforce = pd.read_excel(inforce_file)
inforce['ValDt'] = pd.to_datetime(inforce['ValDt']).dt.date
inforce['HedgeDate'] = pd.to_datetime(inforce['HedgeDate']).dt.date
inforce['Maturity'] = inforce.apply(lambda row: mktdata.get_maturity(row['ValDt'], row['Mat(Yrs)']), axis=1)
inforce['IdxLvl_HedgeDate'] = inforce.apply(lambda row: mktdata.get_px(row['HedgeDate'], row['Bbg_Idx']), axis=1)

# inforce.rename(columns={'AV_Amt': 'Notional'}, inplace=True)

In [24]:
inforce

,Product,Series,Opt_Type,Choice,Index,Bbg_Idx,Mat(Yrs),Mth,HedgeDate,ValDt,Notional,Buffer,Part,Cap,Maturity,IdxLvl_HedgeDate
0,IUL,151,Call Spread,False,MXEA,MXEA Index,1,1,2023-01-20,2023-01-20,"1,849,167.67",1.00,1.00,0.07,2024-01-19,"2,080.41"
1,IUL,151,Call Spread,False,RTY,RTY Index,1,1,2023-01-20,2023-01-20,"2,235,171.72",1.00,1.00,0.07,2024-01-19,"1,867.34"
2,IUL,151,Call Spread,False,SPX,SPX Index,1,1,2023-01-20,2023-01-20,"17,279,589.81",1.00,1.00,0.07,2024-01-19,"3,972.61"
3,IUL,151,Call Spread,False,SPX,SPX Index,3,1,2023-01-20,2023-01-20,"2,510,920.25",1.00,1.00,0.25,2026-01-20,"3,972.61"
4,IUL,153,Call Spread,False,MXEA,MXEA Index,1,1,2023-01-20,2023-01-20,"878,451.62",1.00,1.00,0.07,2024-01-19,"2,080.41"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,2023-12-15,0.01,0.10,1.00,0.15,2024-12-13,"4,719.19"
500,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,2023-12-15,0.04,0.15,1.00,0.13,2024-12-13,"4,719.19"
501,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,2023-12-15,0.03,0.20,1.00,0.12,2024-12-13,"4,719.19"
502,VUL,MSO2,DD,False,SPX,SPX Index,1,12,2023-12-15,2023-12-15,0.01,0.10,1.00,0.12,2024-12-13,"4,719.19"


In [17]:
inforce.dtypes

Product      object
Series       object
Strategy     object
Choice         bool
Index        object
Bbg_Idx      object
Mat(Yrs)      int64
Mth           int64
ValDt        object
AV_Amt      float64
Buffer      float64
Part        float64
Cap         float64
dtype: object

In [ ]:
# Converts current index into a column named 'index' and creates a new index to replace the old
inforce.reset_index(inplace=True)

In [ ]:
inforce.rename(columns={'index': 'Liab_ID'}, inplace=True)

In [14]:
df

,Liab_ID,Product,Series,Strategy,Choice,Index,Bbg_Idx,Mat(Yrs),Mth,ValDt,AV_Amt,Buffer,Part,Cap
0,0,IUL,151,Call Spread,False,MXEA,MXEA Index,1,1,2023-01-20,"1,849,167.67",1.00,1.00,0.07
1,1,IUL,151,Call Spread,False,RTY,RTY Index,1,1,2023-01-20,"2,235,171.72",1.00,1.00,0.07
2,2,IUL,151,Call Spread,False,SPX,SPX Index,1,1,2023-01-20,"17,279,589.81",1.00,1.00,0.07
3,3,IUL,151,Call Spread,False,SPX,SPX Index,3,1,2023-01-20,"2,510,920.25",1.00,1.00,0.25
4,4,IUL,153,Call Spread,False,MXEA,MXEA Index,1,1,2023-01-20,"878,451.62",1.00,1.00,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,499,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,0.01,0.10,1.00,0.15
500,500,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,0.04,0.15,1.00,0.13
501,501,VUL,MSO2,Call Spread,False,SPX,SPX Index,1,12,2023-12-15,0.03,0.20,1.00,0.12
502,502,VUL,MSO2,DD,False,SPX,SPX Index,1,12,2023-12-15,0.01,0.10,1.00,0.12
